# 웨더아이 크롤링
* 웨더아이: 과거 날씨를 볼 수 있는 사이트 (https://www.weatheri.co.kr/bygone/bygone01.php)
* https://idnstory.atlassian.net/wiki/spaces/~327709842/pages/2418540549
## 설정 필요
* 지점 선택: def make_jijum_table 보고 원하는 지점 선택해야함
* 기간 설정

In [2]:
from datetime import datetime, timedelta, date
import pandas as pd

import urllib
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By

from time import sleep

In [16]:
browser2 = webdriver.PhantomJS()

/home/ahkim/env/IM_DA_v1/lib/python3.6/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '


In [17]:
browser2.get('https://www.weatheri.co.kr/bygone/bygone01.php')

In [3]:
def make_jijum_table():
    url2 = 'https://www.weatheri.co.kr/bygone/bygone01.php'

    # get html
    browser2 = webdriver.PhantomJS()
    browser2.get(url2)

    select_location = browser2.find_element_by_name('jijum_id')
    options = select_location.find_elements_by_tag_name("option")

    jijum_table = pd.DataFrame(columns=['id', 'name'])
    for element in options:
        location_id = element.get_attribute("value")
        location_name = element.text
        jijum_info = pd.Series([location_id, location_name], index=['id', 'name'])
        jijum_table = jijum_table.append(jijum_info, ignore_index=True)
    return jijum_table

In [4]:
def make_dt_info(date):
    
    year = date.year
    month = date.month
    day = date.day
    
    return [year, month, day]

In [5]:

def get_data(s_dt_info, e_dt_info, location_id):

    url = 'https://www.weatheri.co.kr/bygone/pastDB_tmp.php?jijum_id={}&start={}-{}-{}&end={}-{}-{}'.\
    format(location_id,
           s_dt_info[0], s_dt_info[1], s_dt_info[2], 
           e_dt_info[0],  e_dt_info[1],  e_dt_info[2]
           )

    # get html
    browser = webdriver.PhantomJS()
    browser.get(url)

    table = browser.find_elements_by_tag_name('table')[2]

    data_list = []
    # table = browser.find_elements_by_tag_name('table')[2]
    tbody = table.find_element_by_tag_name("tbody")
    for i in range(1, len(tbody.find_elements_by_tag_name("tr"))):
        rows = tbody.find_elements_by_tag_name("tr")[i]
        body= rows.find_elements_by_tag_name("td")
        row_data = []
        for index, value in enumerate(body):
            value_data = value.text
            row_data.append(value_data)
        data_list.append(row_data)

    df = pd.DataFrame(data_list)

    header = df.iloc[0]
    df = df[1:]

    df.rename(columns=header, inplace=True)
    df['날짜'] = df['날짜'].str.replace('\n', repl=r'')
    df['날짜'] = df['날짜'].str.split(' ').str[0]
    df['날짜'] = pd.to_datetime(df['날짜'], format='%Y년%m월%d일')
    
    return df

In [6]:
jijum_table = make_jijum_table()
loc_id = int(jijum_table[jijum_table['name'] == '서울']['id'].iloc[0])
print(loc_id)

/home/ahkim/env/IM_DA_v1/lib/python3.6/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '


108


In [7]:
dt_index = pd.date_range(start='20200101', periods=3, freq='MS')


In [8]:
start_dt_info = make_dt_info(dt_index[0])
end_dt_info = make_dt_info(dt_index[-1])

In [9]:
wt_data = pd.DataFrame()
for i in range(0, len(dt_index)):
    start_dt_info = make_dt_info(dt_index[i])
    print(start_dt_info)
    monthly_wt_data = get_data(start_dt_info, end_dt_info, loc_id)
    wt_data = wt_data.append(monthly_wt_data)

[2020, 1, 1]


/home/ahkim/env/IM_DA_v1/lib/python3.6/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '


[2020, 2, 1]


/home/ahkim/env/IM_DA_v1/lib/python3.6/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '


[2020, 3, 1]


/home/ahkim/env/IM_DA_v1/lib/python3.6/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '


In [10]:
wt_data.head()


,날짜,평균 기온,최고 기온,최저 기온,강수량(mm),신적설(cm),평균 풍속(m/s),평균 습도 (%),운량(1/10),일조 시간 (Hr),날씨
1,2020-01-01,-2.2,0.3,-6.5,0.1,0.0,1.2,64,8.9,0.8,흐림/눈/안개
2,2020-01-02,1.0,3.8,-0.7,-,-,1.9,65,7.9,0.0,구름많음/안개
3,2020-01-03,-0.1,4.6,-3.4,-,-,1.8,57,0.0,8.8,맑음/안개
4,2020-01-04,1.1,6.1,-2.8,-,-,1.9,51,2.1,7.9,구름조금/안개
5,2020-01-05,1.3,6.6,-3.2,-,-,1.1,46,3.9,7.1,구름조금/안개


In [11]:

url2 = 'https://www.weatheri.co.kr/bygone/bygone01.php'

# get html
browser2 = webdriver.PhantomJS()
browser2.get(url2)

select_location = browser2.find_element_by_name('jijum_id')
options = select_location.find_elements_by_tag_name("option")

jijum_table = pd.DataFrame(columns=['id', 'name'])
for element in options:
    location_id = element.get_attribute("value")
    location_name = element.text
    jijum_info = pd.Series([location_id, location_name], index=['id', 'name'])
    jijum_table = jijum_table.append(jijum_info, ignore_index=True)

/home/ahkim/env/IM_DA_v1/lib/python3.6/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '


In [12]:
jijum_table.head()


,id,name
0,105,강릉
1,259,강진군
2,201,강화
3,294,거제
4,284,거창


In [13]:
url = 'https://www.weatheri.co.kr/bygone/pastDB_tmp.php?jijum_id=108&start=2021-03-01&end=2021-05-31'

# get html
browser = webdriver.PhantomJS()
browser.get(url)

table = browser.find_elements_by_tag_name('table')[2]

data_list = []
# table = browser.find_elements_by_tag_name('table')[2]
tbody = table.find_element_by_tag_name("tbody")
for i in range(1, len(tbody.find_elements_by_tag_name("tr"))):
    rows = tbody.find_elements_by_tag_name("tr")[i]
    body= rows.find_elements_by_tag_name("td")
    row_data = []
    for index, value in enumerate(body):
        value_data = value.text
        row_data.append(value_data)
    data_list.append(row_data)

df = pd.DataFrame(data_list)

header = df.iloc[0]
df = df[1:]

df.rename(columns=header, inplace=True)
df['날짜'] = df['날짜'].str.replace('\n', repl=r'')
df['날짜'] = df['날짜'].str.split(' ').str[0]
df['날짜'] = pd.to_datetime(df['날짜'], format='%Y년%m월%d일')

/home/ahkim/env/IM_DA_v1/lib/python3.6/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '


In [14]:
df.head()


,날짜,평균 기온,최고 기온,최저 기온,강수량(mm),신적설(cm),평균 풍속(m/s),평균 습도 (%),운량(1/10),일조 시간 (Hr),날씨
1,2021-03-01,4.7,9.0,0.5,67.5,2.3,3.4,91,10.0,0.0,흐림/눈/비/진눈개비
2,2021-03-02,2.3,6.9,0.2,0.1,0.0,1.8,76,5.5,4.7,구름많음/눈
3,2021-03-03,4.4,9.9,-1.2,-,-,1.8,64,4.5,10.4,구름조금
4,2021-03-04,7.2,13.0,2.5,-,-,2.0,62,6.2,1.9,구름많음
5,2021-03-05,8.9,16.2,3.0,-,-,2.0,70,1.2,10.5,맑음
